In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import sampler, TensorDataset, Dataset

from sklearn import metrics
import itertools
import math
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from imgaug import augmenters as iaa
import imgaug as ia
import pandas as pd
from pandas import Series
import random
import scipy
import time
import cv2
from PIL import Image, ImageOps, ImageEnhance

from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_absolute_error, r2_score

# DATA GENERATE

In [2]:
w = 299
h = 299

In [3]:
def img_crop(img, w, h):
    img_h = img.shape[0]
    img_w = img.shape[1]
    start_h, start_w = random.randint(0, int(img_h - h)), random.randint(0, int(img_w - w))
    return img[start_h:start_h+h, start_w:start_w+w, :]

In [4]:
# image pre-process
# mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]
def img_prepro(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (w,h))
#     img = img_crop(img, w, h)
    img = img/255.
    img[:, :, 0] = (img[:, :, 0] - 0.485)/0.229
    img[:, :, 1] = (img[:, :, 1] - 0.456)/0.224
    img[:, :, 2] = (img[:, :, 2] - 0.406)/0.225
    img = np.swapaxes(np.swapaxes(img, 0, 1), 0, 2)
    return img

In [5]:
def data_generation(train_id):
    X = []
    Y = []
    for i in train_id:
        path = df_r[df_r['uid_date'] == i]['path']
        for p in path:
            try:
                X.append(img_prepro(p))
                Y.append(df_r[df_r['path'] == p]['target'].values[0])
            except:
                print(p)
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [6]:
# read full df
df = pd.read_csv('/data/put_data/timliu/kidney/pro_img/VBTEC/clahe_right_label2/df_with_labels.csv', index_col = False)
# df = pd.DataFrame([row for index, row in df.iterrows() if 'GE' in row['path']])
# df2 = pd.read_csv('INFINITT_final_1.csv', index_col = False)
# df3 = pd.read_csv('VBTEC_final.csv', index_col = False)
# df = pd.concat((df, df2, df3), axis = 0)
# df = df.reset_index(drop = True)
df.scr = df.scr*10
print(df.shape)
# label classification
# df['target'] = 0
# for index, row in df.iterrows():
#     eg = float(row['egfr_mdrd'])
#     if eg >= 30:
#         label = 0
#     else:
#         label = 1
#     df.iloc[index, 16] = int(label)
df = df[df.dm == 0].reset_index()
# df_r = df[(df.length != 'Suspension') & (df.dm == 0)]
# df_r.length = df_r.length.astype('float')

# df_r = df_r[((df_r.egfr_mdrd >= 15) & (df_r.length <= 10)) | ((df_r.egfr_mdrd <= 90) & (df_r.length >= 8))]

# df_r = df_r.reset_index()
# df = df_r
# print(df.shape)
# df.length = df.length.astype('float')
print(df.shape)
df.head()

(9900, 15)
(9900, 16)


,index,path,type,uid,chtno,name,contentdate,year,uid_date,scr,egfr_mdrd,dm,cvd,htn,sex,age
0,0,/data/put_data/timliu/kidney/pro_img/VBTEC/cla...,right,5377_I00068073416,5377,I00068073416,20080716,2008,5377_20080716,38.0,19.564004,0,0,0,0,33
1,1,/data/put_data/timliu/kidney/pro_img/VBTEC/cla...,right,5377_I00068073364,5377,I00068073364,20080716,2008,5377_20080716,38.0,19.564004,0,0,0,0,33
2,2,/data/put_data/timliu/kidney/pro_img/VBTEC/cla...,right,2014_I00104545140,2014,I00104545140,20120828,2012,2014_20120828,12.4,48.701118,0,1,1,1,45
3,3,/data/put_data/timliu/kidney/pro_img/VBTEC/cla...,right,2014_I00104545135,2014,I00104545135,20120828,2012,2014_20120828,12.4,48.701118,0,1,1,1,45
4,4,/data/put_data/timliu/kidney/pro_img/VBTEC/cla...,right,2014_I00104545139,2014,I00104545139,20120828,2012,2014_20120828,12.4,48.701118,0,1,1,1,45


In [7]:
# for index, row in df.iterrows():
#     df.iloc[index, 1] = '/data/put_data/timliu/kidney/pro_img/GE/' + 'removed_' + df.iloc[index, 1].split('/')[-2] + '/' + df.iloc[index, 1].split('/')[-1]

In [8]:
# from statsmodels.distributions.empirical_distribution import ECDF
# ecdf = ECDF(df.egfr_mdrd.values)
# plt.plot(ecdf.x, ecdf.y)
# plt.show()
# plt.hist(df.egfr_mdrd.values, bins=50)
# plt.show()
# plt.close()

In [9]:
# unique_id = list(set([row['uid_date'] for index, row in df.iterrows()]))
# one_kidney = []
# for i in unique_id:
#     df_test = df[df['uid_date'] == i]['type']
#     if len(set(df_test)) < 2:
#         one_kidney.append(i)
        
# df = pd.DataFrame([row for index, row in df.iterrows() if row['uid_date'] not in one_kidney])
# df = df.reset_index(drop = True)
# print(df.shape)

In [10]:
# split data every-step = 10 
ratio = 0.9
random.seed(45) #78
train_id = []
test_id = []
for i in range(0, 34):
    df_sub = df[df['egfr_mdrd'].between(5*i, 5*(i+1), inclusive = (True, False))]
    d_list = Series.tolist(df_sub['uid_date'])
    d_list = list(set(d_list))
    random.shuffle(d_list)
    train_id += d_list[:int(ratio*len(d_list))]
    test_id += d_list[int(ratio*len(d_list)):]
print('all:', len(set(df.egfr_mdrd)), 'train:', len(train_id), 'test:', len(test_id))

all: 4685 train: 4197 test: 485


In [11]:
# splitting data 
# ratio = 0.8
# unique_id = list(set(df['uid_date']))
# random.shuffle(unique_id)
# train_id = unique_id[:int(ratio*len(unique_id))]
# test_id = unique_id[int(ratio*len(unique_id)):]
# print(len(train_id))
# print(len(test_id))

# START

In [12]:
torch.cuda.set_device(0)
# specify dtype
use_cuda = torch.cuda.is_available()
print(use_cuda)
if use_cuda:
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor
print(torch.cuda.device_count())

True
2


In [13]:
# Hyperparameters
batch_size = 64
learning_rate = 1e-6
momentum = 0.9
num_epoch = 300

In [14]:
# class kidney_Dataset(Dataset):
#     def __init__(self, data_tensor1, target_tensor):
#         assert data_tensor1.size(0) ==  target_tensor.size(0)
#         self.data_tensor1 = data_tensor1
#         self.target_tensor = target_tensor

#     def __getitem__(self, index):
#         return self.data_tensor1[index], self.target_tensor[index]

#     def __len__(self):
#         return self.data_tensor1.size(0)

In [15]:
# train_dataset = kidney_Dataset(torch.from_numpy(train_X), torch.from_numpy(train_Y))
# test_dataset = kidney_Dataset(torch.from_numpy(test_X), torch.from_numpy(test_Y))
# trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
# testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [16]:
sometimes = lambda aug: iaa.Sometimes(0.9, aug)
seq_1 = iaa.Sequential([
    sometimes(iaa.Affine(
        rotate=(-25, 25),
        scale=(1, 1)
    ))
])

seq_2 = iaa.Sequential([
    iaa.Affine(rotate=(-10, 10))
])

In [17]:
# center crop
transformations_1 = transforms.Compose([transforms.CenterCrop([450, 450])])
# normalize
transformations_2 = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))]) # , transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

In [18]:
def img_process_transform(img, augmentor, transformation, transform):
    img = np.expand_dims(img, axis = 0)
    if transform:
#         augmentor_det = augmentor.to_deterministic()
        img = augmentor.augment_images(img)
    img = np.squeeze(img, axis = 0)
    img = Image.fromarray(img.astype('uint8'), mode="RGB")
    img = transformation(img)
    return img

In [19]:
from sklearn import preprocessing
from skimage.filters import gaussian
size = 224, 224
def img_process_PIL(path, transform):
    # random room and central crop
    img = cv2.imread(path) 
    b, g, r = cv2.split(img)
    img = cv2.merge((r, g, b))
    img = img_process_transform(img, seq_1, transformations_1, transform)
    img.thumbnail(size, Image.ANTIALIAS)
#     if transform:
    img = img_process_transform(img, seq_2, transformations_2, transform)
    return img

In [20]:
# img = img_process_PIL(df['path'].values[1700], transform = False)
# img
# img.size
# plt.imshow(img)
# plt.show()
# plt.close()

In [21]:
class kidney_Dataset(Dataset):
    def __init__(self, train_id, df, transform = True):
        self.train_id = train_id
        self.df = df
        self.transform = transform
    def __getitem__(self, index):
        sample_count = len(self.df[self.df['uid_date'] == self.train_id[index]]['path'].values)
        select_index = random.randint(0, sample_count-1)
        path = self.df[self.df['uid_date'] == self.train_id[index]]['path'].values[select_index]
        img = img_process_PIL(path, self.transform)
#         length = self.df[self.df['uid_date'] == self.train_id[index]]['length'].values
#         max_ind = np.where(length == np.max(length))[0][0]
#         length = np.atleast_1d(length[max_ind])
        sex = np.atleast_1d(self.df[self.df['uid_date'] == self.train_id[index]]['sex'].values[select_index])
        age = np.atleast_1d((self.df[self.df['uid_date'] == self.train_id[index]]['age'].values[select_index] - np.mean(self.df.age)) / np.std(self.df.age))
        cvd = np.atleast_1d(self.df[self.df['uid_date'] == self.train_id[index]]['cvd'].values[select_index])
        htn = np.atleast_1d(self.df[self.df['uid_date'] == self.train_id[index]]['htn'].values[select_index])
        label = self.df[self.df['uid_date'] == self.train_id[index]]['egfr_mdrd'].values[select_index]
        return img,sex,age,cvd,htn,label

    def __len__(self):
        return len(self.train_id)

In [22]:
train_dataset = kidney_Dataset(train_id, df, transform = False)
test_dataset = kidney_Dataset(test_id, df, transform = False)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [23]:
# dataiter = iter(trainloader)
# images1,sex,age,cvd,htn, labels = dataiter.next()
# print(images1.shape)

In [24]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.base_model = models.vgg16(pretrained=True).type(dtype)
#         self.left_model = nn.Sequential(*list(copy.deepcopy(self.base_model).features.children()))
#         self.left_model.conv1 = nn.Conv2d(1, 64, kernel_size = (7, 7), stride=(2, 2), padding = (3,3), bias = False).type(dtype)
#         self.left_model.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias = False).type(dtype)
        self.fc = nn.Sequential(
            nn.Linear(1004, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
        
    def forward(self, x1, sex, age, cvd, dm):
        out2 = self.base_model(x1)
        out2 = out2.view(out2.size(0), -1)
        out = torch.cat((out2, sex, age, cvd, dm), 1)
        out = self.fc(out)
        return out

In [25]:
# define model
model = CNN().type(dtype)
# model = models.resnet101(pretrained=False).type(dtype)
# change input size
# model.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1)).type(dtype)
# model.conv1 = nn.Conv2d(1, 64, kernel_size = (3, 3), stride=(2, 2), padding = (3,3), bias = False).type(dtype)
# model.fc = nn.Linear(2048, 1)
# model.classifier._modules['6'] = nn.Linear(4096, 5).type(dtype)
# model.fc = nn.Sequential(nn.Linear(2048, 1)).type(dtype)
# model.classifier._modules['7'] = nn.Dropout(p = 0.5).type(dtype)
# model.classifier._modules['8'] = nn.Linear(1000, 1).type(dtype)
net = torch.nn.DataParallel(model)

In [26]:
# model.classifier = nn.Sequential(nn.Linear(2208, 1000), 
#                                 nn.ReLU(),
#                                 nn.Linear(1000, 1)
#                                 ).type(dtype)
# net = torch.nn.DataParallel(model)

In [27]:
# # # # test
# t1 = Variable(torch.rand(64, 3, 224, 224)).type(dtype)
# lengtht = Variable(torch.rand(64, 1)).type(dtype)
# sext = Variable(torch.rand(64, 1)).type(dtype)
# aget = Variable(torch.rand(64, 1)).type(dtype)
# dmt = Variable(torch.rand(64, 1)).type(dtype)
# net(t1,lengtht,sext,aget,dmt)

In [28]:
def plot_losses(loss_history1=None, loss_history2=None):
    plt.clf()
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    if loss_history1:
        ax1.plot(loss_history1, color="blue", label="train")
    if loss_history2:
        ax1.plot(loss_history2, color="orange", label="test")
    #ax2 = ax1.twinx()
    #ax2.set_yscale('log')
    plt.xlabel("epoch") 
    plt.ylabel("loss") 
    plt.legend(bbox_to_anchor=(0.8, 0.9), loc=2, borderaxespad=0.)
    plt.title("loss")
    plt.savefig(model_dir + 'output_losses.png')
    plt.close()

In [29]:
# loss and optimizer
# criterion = nn.SmoothL1Loss()
criterion = nn.MSELoss(size_average=False)
criterion2 = nn.L1Loss(size_average=False)
# criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-6)
# optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, nesterov=True, weight_decay=1e-6)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=10,verbose=True)
# model dir
time_str = time.strftime("%m%d")
model_dir = 'model_for_prediction/' + time_str + '/rgb_clahe_vbtec_vgg16_single_removedm_removelabel/' 
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
print('model will be saved to ', model_dir)

model will be saved to  model_for_prediction/0306/rgb_clahe_vbtec_vgg16_single_removedm_removelabel/


In [ ]:
# for i, tdata in enumerate(testloader, 0):
#     tinputs1, tsex, tage, tcvd, tdm, tlabels = tdata
#     tinputs1, tsex, tage, tcvd, tdm, tlabels = Variable(tinputs1, volatile=True).type(dtype), Variable(tsex, volatile=True).type(dtype), Variable(tage, volatile=True).type(dtype), Variable(tcvd, volatile=True).type(dtype), Variable(tdm, volatile=True).type(dtype), Variable(tlabels, volatile=True).type(dtype)

In [ ]:
train_loss = []
test_loss_l1 = []
test_loss = []
patience = 0
best_loss = float('Inf')

# start training
for epoch in range(num_epoch):  # loop over the dataset multiple times
    running_loss1 = []
    running_loss2 = []
    running_loss3 = []
    tStart = time.time()

    for i, data in enumerate(trainloader, 0):
        net.train(True)    
        # get the inputs
        inputs1, sex, age, cvd, dm, labels = data
        # wrap them in Variable
        inputs1, sex, age, cvd, dm, labels = Variable(inputs1).type(dtype),\
                                             Variable(sex).type(dtype),\
                                             Variable(age).type(dtype),\
                                             Variable(cvd).type(dtype),\
                                             Variable(dm).type(dtype),\
                                             Variable(labels).type(dtype)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs1, sex, age, cvd, dm)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        # loss
        running_loss1.append(loss.data[0])
        
    for i, tdata in enumerate(testloader, 0):
        net.train(False)
        tinputs1, tsex, tage, tcvd, tdm, tlabels = tdata
        tinputs1, tsex, tage, tcvd, tdm, tlabels = Variable(tinputs1, volatile=True).type(dtype),\
                                                   Variable(tsex, volatile=True).type(dtype),\
                                                   Variable(tage, volatile=True).type(dtype),\
                                                   Variable(tcvd, volatile=True).type(dtype),\
                                                   Variable(tdm, volatile=True).type(dtype),\
                                                   Variable(tlabels, volatile=True).type(dtype) 
        toutput = net(tinputs1, tsex, tage, tcvd, tdm)
        tloss = criterion(toutput, tlabels)
        tloss_l1 = criterion2(toutput, tlabels)
        running_loss2.append(tloss.data[0])
        running_loss3.append(tloss_l1.data[0])
    
    train_loss_epoch, test_loss_epoch, test_loss_epoch_l1 = np.sum(running_loss1) / len(train_id),\
                                                            np.sum(running_loss2) / len(test_id),\
                                                            np.sum(running_loss3) / len(test_id)
    scheduler.step(test_loss_epoch)
    
    train_loss.append(train_loss_epoch)
    test_loss.append(test_loss_epoch)
    test_loss_l1.append(test_loss_epoch_l1)
    
    tEnd = time.time()
    
    #statistics
    print('Epoch {}/{}, Train Loss: {:.3f}, Test Loss: {:.3f}, Test Loss l1: {:.3f}, Time: {:.3f}'.format(epoch+1, num_epoch, train_loss[-1], test_loss[-1], test_loss_l1[-1],(tEnd-tStart)))


    #early stopping
    if epoch > 20:
        if test_loss[-1] < best_loss:
            best_loss = test_loss[-1]
            torch.save(net.state_dict(), model_dir + 'model.pkl')
            patience = 0
        else:
            patience += 1
            
        if patience > 50:
            print('Early stopping')
            break
        # loss history 
    if epoch % 10 == 0 and epoch != 0:
        plot_losses(train_loss, test_loss)
    
print('Finished Training')

Epoch 1/300, Train Loss: 1810.043, Test Loss: 2046.839, Test Loss l1: 33.791, Time: 46.018
Epoch 2/300, Train Loss: 1358.428, Test Loss: 1058.109, Test Loss l1: 23.282, Time: 42.204
Epoch 3/300, Train Loss: 724.286, Test Loss: 913.529, Test Loss l1: 22.866, Time: 42.328
Epoch 4/300, Train Loss: 703.840, Test Loss: 888.829, Test Loss l1: 22.892, Time: 42.553
Epoch 5/300, Train Loss: 699.078, Test Loss: 883.816, Test Loss l1: 22.641, Time: 42.638
Epoch 6/300, Train Loss: 683.323, Test Loss: 877.188, Test Loss l1: 22.666, Time: 42.770
Epoch 7/300, Train Loss: 673.381, Test Loss: 874.466, Test Loss l1: 22.507, Time: 42.234
Epoch 8/300, Train Loss: 672.020, Test Loss: 858.858, Test Loss l1: 22.493, Time: 42.219
Epoch 9/300, Train Loss: 662.368, Test Loss: 861.471, Test Loss l1: 22.458, Time: 42.802
Epoch 10/300, Train Loss: 650.739, Test Loss: 845.958, Test Loss l1: 22.029, Time: 42.650
Epoch 11/300, Train Loss: 642.737, Test Loss: 840.281, Test Loss l1: 22.158, Time: 42.792
Epoch 12/300, T

In [ ]:
# load model 
net.load_state_dict(torch.load(model_dir+ 'model.pkl'))

In [ ]:
plot_losses(train_loss, test_loss)

## Classification evaluation

In [ ]:
# def class_generator(dataloader, net):
#     true = []
#     pred = []
#     for i, data in enumerate(dataloader):
#         sample, label = data
#         sample, label = Variable(sample, volatile = True).type(dtype), Variable(label, volatile = True).type(torch.cuda.LongTensor)
#         output = net(sample)
#         _, predicted = torch.max(output.data, 1)
#         true.extend(label.data.cpu().numpy())
#         pred.extend(predicted.cpu().numpy())
#     true = np.asarray(true)
#     pred = np.asarray(pred)
#     return true, pred

In [ ]:
# true, pred = class_generator(testloader, net.eval())
# true = np.squeeze(true, axis=1)

In [ ]:
# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     fmt = '.2f' if normalize else 'd'
#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, format(cm[i, j], fmt),
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

In [ ]:
# from sklearn import metrics
# import itertools
# print('Accuracy: {:.3f}%'.format(metrics.accuracy_score(true, pred)*100))
# unique, counts = np.unique(true, return_counts=True)
# print('Counts of each labels: ', dict(zip(unique, counts)))
# cm = metrics.confusion_matrix(true, pred)
# plot_confusion_matrix(cm, classes = np.sort(list(set(true))))

## Regression evaluation

In [ ]:
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
def plot_making(true, pred, types):
    cor = pearsonr(true, pred)[0]
    mae = mean_absolute_error(true, pred)
    r2 = r2_score(true, pred) 
    plt.figure(0)
    plt.scatter(true, pred, alpha = .15, s = 20)
    plt.xlabel('True_Y')
    plt.ylabel('Pred_Y')
    plt.title(" data \n" + "MAE = %4f; Cor = %4f; R2 = %4f; #samples = %d" % (mae, cor, r2, len(true)))
    plt.savefig(model_dir + types + "_plot_scatter.png" , dpi = 200)
    plt.show()
    plt.close()

In [ ]:
def loss_generator(testloader, model, types):
    test_loss = []
    true = []
    pred = []
    for i, tdata in enumerate(testloader, 0):
        tinputs1, tsex, tage, tcvd, tdm, tlabels = tdata
        tinputs1, tsex, tage, tcvd, tdm, tlabels = Variable(tinputs1, volatile=True).type(dtype), Variable(tsex, volatile=True).type(dtype), Variable(tage, volatile=True).type(dtype), Variable(tcvd, volatile=True).type(dtype), Variable(tdm, volatile=True).type(dtype), Variable(tlabels, volatile=True).type(dtype)
        toutput = net(tinputs1, tsex, tage, tcvd, tdm)
        tloss = criterion(toutput, tlabels)
        test_loss.append(tloss.data[0])
        true.extend(tlabels.data.cpu().numpy())
        pred.extend(toutput.data.cpu().numpy())
    true = np.asarray(true)
    true = np.expand_dims(true, axis=1)
    pred = np.asarray(pred)
    plot_making(true, pred, types = types)
    print('L1 Loss on images: %r' % (np.sum(test_loss)/pred.shape[0]))
    
train_dataset = kidney_Dataset(train_id, df, transform = False)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
loss_generator(testloader, net.eval(), types = 'test')
loss_generator(trainloader, net.eval(), types = 'train')

In [ ]:
%%javascript
Jupyter.notebook.session.delete();